In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV,train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report,make_scorer,precision_recall_curve,average_precision_score
from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.ensemble import RandomForestClassifier



In [2]:
# load the data

data = pd.read_csv("../processed_data/merged_data.csv")

list_bp = ['avg_dbp', 'avg_diff', 'avg_sbp', 'max_sbp']
list_ed = ['age', 'sex', 'language', 'insurance_type', 'primary_care', 
            'ed_name', 'bpa_response', 'htn_on_pl', 'htn_on_pmh', 
            'hld_on_pl', 'hld_on_pmh', 'family_dm', 'tobacco_user', 
            'htn_meds', 'statin_meds', 'disposition', 'detailed_race', 
            'weight', 'bmi', 'hba1c', 'height', 'sbp_1st', 'dbp_1st', 
            'poct_gluc']
list_lab = ['max_value_GLUCOSE', 'avg_value_GLUCOSE', 'max_value_CREATININE', 
            'min_value_CREATININE', 'min_value_GLUCOSE',  'avg_value_CREATININE', 
            'avg_value_HEMOGLOBIN A1C', 'max_value_HEMOGLOBIN A1C', 'min_value_HEMOGLOBIN A1C',  
            'min_value_GLUCOSE, POC', 'avg_value_GLUCOSE, POC', 'max_value_GLUCOSE, POC']
list_geo = [
    'po_box', 'homeless', 'total_pop', 'households', 'housing_units', 
    'p_children', 'p_elderly', 'p_adults', 'p_female', 'mdn_age', 
    'p_nhwhite', 'p_nhblack', 'p_hispanic', 'p_nhasian', 'p_other', 
    'p_moved', 'p_longcommute', 'p_marriednone', 'p_marriedkids', 
    'p_singlenone', 'p_malekids', 'p_femalekids', 'p_cohabitkids', 
    'p_nohsdeg', 'p_hsonly', 'p_somecollege', 'p_collegeplus', 
    'p_onlyenglish', 'p_spanishlimited', 'p_asianlimited', 'p_otherlimited', 
    'p_limitedall', 'p_notlimited', 'p_popbelow1fpl', 'p_popbelow2fpl', 
    'p_povmarriedfam', 'p_povmalefam', 'p_povfemalefam', 'hh_mdnincome', 
    'p_pubassist', 'p_foodstamps', 'p_assistorfood', 'p_unemployed', 
    'h_vacant', 'h_renter', 'h_occupants', 'h_novehicles', 'h_mdnrent', 
    'h_rentpercent', 'h_houseprice', 'p_private', 'p_medicare', 'p_medicaid', 
    'p_otherinsur', 'p_uninsured', 'h_nointernet', 'h_nocomputer', 
    'p_foreign', 'p_disabled']
list_visit = ['visit_type']


lists = list_bp+ list_ed+ list_lab+ list_geo+ list_visit
X_all = data[lists]
y = data['pcp_followup'].map({'Yes': 1, 'No': 0})
y = np.array(y).astype(int)

In [3]:
# encode

numeric_cols = X_all.select_dtypes(include=['number']).columns
categorical_cols = X_all.select_dtypes(exclude=['number']).columns
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_cols),
            ('cat', categorical_transformer, categorical_cols)
        ]
    )

X_preprocessed = preprocessor.fit_transform(X_all)
if hasattr(X_preprocessed, "toarray"):
    X_preprocessed = X_preprocessed.toarray()


numeric_feature_names = numeric_cols 
cat_feature_names = preprocessor.named_transformers_['cat']['onehot'].get_feature_names_out(categorical_cols)
all_feature_names = list(numeric_feature_names) + list(cat_feature_names)

In [4]:
# selected by importance
# select tail 10 features
X_train_val, X_test, y_train_val, y_test = train_test_split(X_preprocessed, y, test_size=0.2, random_state=50)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=50)

lasso = Lasso(alpha=1e-3,
              max_iter=1035,
              tol=1e-10,
              random_state=50,
              selection='cyclic',)
lasso.fit(X_train, y_train)

coef = lasso.coef_

feature_importance = pd.DataFrame({
    'feature': all_feature_names,
    'importance': coef
})

feature_importance = feature_importance.sort_values('importance', ascending=False)

selected_feature_names = feature_importance[feature_importance['importance'] < 0]
selected_feature_names = selected_feature_names.tail(10)
X_train = pd.DataFrame(X_train, columns=all_feature_names)[selected_feature_names['feature']]
X_val = pd.DataFrame(X_val, columns=all_feature_names)[selected_feature_names['feature']]
X_test= pd.DataFrame(X_test, columns=all_feature_names)[selected_feature_names['feature']]

In [5]:
param_grid = {
    'criterion': ['gini', 'entropy'],
    'n_estimators': [100, 200, 500],
    'max_depth': [5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2'],
    'class_weight': ['balanced', None]
}

scorer = make_scorer(accuracy_score, average='macro')

grid_search = GridSearchCV(
    RandomForestClassifier(random_state=50), 
    param_grid, 
    cv=5,  
    scoring=scorer, 
    n_jobs=-1  
)

grid_search.fit(X_train, y_train)
print(grid_search.best_params_)
best_model = grid_search.best_estimator_

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}


In [6]:
y_proba = best_model.predict_proba(X_val)[:, 1]
precision, recall, thresholds = precision_recall_curve(
    y_val, 
    y_proba,
    pos_label=1)

average_accuracies  = []
for threshold in thresholds:
    y_pred = (y_proba >= threshold).astype(int)
    acc_0 = np.mean(y_pred[y_val == 0] == 0) 
    acc_1 = np.mean(y_pred[y_val == 1] == 1)  
    avg_acc = (acc_0 + acc_1) / 2
    average_accuracies.append(avg_acc)

best_index = np.argmax(average_accuracies)
optimal_threshold = thresholds[best_index]
print(f"Optimal threshold: {optimal_threshold:.2f}")

Optimal threshold: 0.47


In [7]:
y_val_pred = np.where(y_proba > optimal_threshold, 1, 0)
accuracy = accuracy_score(y_val, y_val_pred)
conf_matrix = confusion_matrix(y_val, y_val_pred)

print(f'Accuracy: {accuracy:.2f}')
print('Confusion Matrix:')
print(conf_matrix)
print(classification_report(y_val, y_val_pred))



Accuracy: 0.71
Confusion Matrix:
[[ 3  9]
 [ 3 26]]
              precision    recall  f1-score   support

           0       0.50      0.25      0.33        12
           1       0.74      0.90      0.81        29

    accuracy                           0.71        41
   macro avg       0.62      0.57      0.57        41
weighted avg       0.67      0.71      0.67        41



In [8]:
# test set
y_proba = best_model.predict_proba(X_test)[:, 1]
precision, recall, thresholds = precision_recall_curve(
    y_test, 
    y_proba,
    pos_label=1)
average_accuracies  = []
for threshold in thresholds:
    y_pred = (y_proba >= threshold).astype(int)
    acc_0 = np.mean(y_pred[y_test == 0] == 0) 
    acc_1 = np.mean(y_pred[y_test == 1] == 1)  
    avg_acc = (acc_0 + acc_1) / 2
    average_accuracies.append(avg_acc)

best_index = np.argmax(average_accuracies)
optimal_threshold = thresholds[best_index]

y_test_pred = np.where(y_proba > optimal_threshold, 1, 0)
accuracy = accuracy_score(y_test, y_test_pred)

conf_matrix = confusion_matrix(y_test, y_test_pred)

print(f'Accuracy: {accuracy:.2f}')
print('Confusion Matrix:')
print(conf_matrix)
print(classification_report(y_test, y_test_pred))


Accuracy: 0.55
Confusion Matrix:
[[ 7  5]
 [18 21]]
              precision    recall  f1-score   support

           0       0.28      0.58      0.38        12
           1       0.81      0.54      0.65        39

    accuracy                           0.55        51
   macro avg       0.54      0.56      0.51        51
weighted avg       0.68      0.55      0.58        51

